In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://09991c9b5d22:7077") \
    .config("spark.jars.packages", "org.neo4j:neo4j-connector-apache-spark_2.13:5.3.10_for_spark_3") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.neo4j#neo4j-connector-apache-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-384061c9-fc4a-46f2-85dc-3f43d70e39f3;1.0
	confs: [default]
	found org.neo4j#neo4j-connector-apache-spark_2.13;5.3.10_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.13_common;5.3.10_for_spark_3 in central
	found org.neo4j#caniuse-core;1.3.0 in central
	found org.neo4j#caniuse-api;1.3.0 in central
	found org.jetbrains.kotlin#kotlin-stdlib;2.1.20 in central
	found org.jetbrains#annotations;13.0 in central
	found org.neo4j#caniuse-neo4j-detection;1.3.0 in central
	found org.neo4j.driver#neo4j-java-driver-slim;4.4.21 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found io.netty#netty-handler;4.1.

In [35]:
from spark_utils import SparkUtils
from pyspark.sql.functions import count

In [44]:
schema = [
    ("from", "string"),
    ("to", "string")
]
spark_schema = SparkUtils.generate_schema(schema)
#i know this is not a good practice when it comes to pyspark i shoudnt direct to a single document, 
# more so when its supposed to take a lot more than just one document
df = spark.read.option("header", True).schema(spark_schema).csv("/opt/spark/work-dir/data/connections.csv")

string
string


In [45]:
#instead of this, i want to create a new coulmn, that counts how many times, or agregates by the column duplicates, as in, how many times is a connection present
#df1 = df.dropDuplicates(['from', 'to'])
df1 = df.groupBy("from", "to").agg(count("*").alias("connection_count"))
finaldf = df1.dropna()

In [46]:
from pyspark.sql.functions import col
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import coalesce, lit

In [47]:
#i drop duplicates twice, so the question is which one would take less time
#Vertices DataFrame:


outgoing_links = finaldf.groupBy("from").agg(count("*").alias("outgoing_count"))
incoming_links = finaldf.groupBy("to").agg(count("*").alias("incoming_count"))


vert_df1 = finaldf.select(col("from").alias("link"))
vert_df2 = finaldf.select(col("to").alias("link"))
vertices_df = reduce(DataFrame.unionAll, [vert_df1, vert_df2]).dropDuplicates(["link"])


vertices_df = vertices_df.join(
    outgoing_links,
    vertices_df.link == outgoing_links["from"],
    "left"
).drop("from")
vertices_df = vertices_df.join(
    incoming_links,
    vertices_df.link == incoming_links["to"],
    "left"
).drop("to")



#failed because we have nulls in edge websites.
vertices_df = vertices_df.withColumn(
    "outgoing_count",
    coalesce(col("outgoing_count"), lit(0))
)
vertices_df = vertices_df.withColumn(
    "incoming_count",
    coalesce(col("incoming_count"), lit(0))
)

In [48]:
# Add the code to write a graph from PySpark's DataFrames to Neo4j
neo4j_url = "bolt://neo4j-iteso:7687"
neo4j_user = "neo4j"
neo4j_passwd = "neo4j@1234"


vertices_df.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("labels", ":URL") \
  .option("node.keys", "link") \
  .option("node.properties", "outgoing_count,incoming_count") \
  .save()

print(f"{vertices_df.count()} URL wrote in Neo4j")


16619 URL wrote in Neo4j


In [41]:
finaldf.show(n=5)

+--------------------+--------------------+----------------+
|                from|                  to|connection_count|
+--------------------+--------------------+----------------+
|https://www.iteso.mx|https://www.iteso...|               2|
|https://www.iteso.mx|https://posgrados...|               1|
|https://www.iteso.mx|https://www.iteso...|               2|
|https://www.iteso.mx|https://mi.iteso....|               1|
|https://www.iteso.mx|https://www.iteso...|               4|
+--------------------+--------------------+----------------+
only showing top 5 rows


In [49]:
#3 minutos
finaldf.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Overwrite") \
  .option("url", neo4j_url) \
  .option("authentication.basic.username", neo4j_user) \
  .option("authentication.basic.password", neo4j_passwd) \
  .option("relationship", "CONNECTION") \
  .option("relationship.save.strategy", "keys") \
  .option("relationship.source.labels", ":URL") \
  .option("relationship.source.save.mode", "match") \
  .option("relationship.source.node.keys", "from:link") \
  .option("relationship.target.labels", ":URL") \
  .option("relationship.target.save.mode", "match") \
  .option("relationship.target.node.keys", "to:link") \
  .option("relationship.properties", "connection_count") \
  .save()

#.option("relationship.source.node.keys", "src:id") format seems to take on column1InRelationship:ColumnInEdge

print(f"{finaldf.count()} edges wrote in Neo4j")

158640 edges wrote in Neo4j
